# e - commerce

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import calendar
import warnings
from operator import attrgetter
import matplotlib.colors as mcolors
import numpy as np

1. Сколько у нас пользователей, которые совершили покупку только один раз?

Если пользователь совершил покупку только один раз, то его айди встретится всего один раз.

In [3]:
customers = pd.read_csv('/home/jupyter-a-berezin-18/Project 1/olist_customers_dataset.csv')

In [4]:
customers

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,3937,sao paulo,SP
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,6764,taboao da serra,SP
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,60115,fortaleza,CE
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,92120,canoas,RS


In [5]:
orders = pd.read_csv('/home/jupyter-a-berezin-18/Project 1/olist_orders_dataset.csv',
                     parse_dates = ['order_purchase_timestamp', 'order_estimated_delivery_date'])

In [6]:
merge = orders.merge(customers, how = 'inner', on = 'customer_id')

In [9]:
merge.value_counts('order_status')

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
dtype: int64

In [6]:
one_order_customers = merge.value_counts('customer_unique_id') \
    .to_frame() \
    .rename(columns={0:'orders'}) \
    .query('orders == 1') \
    .reset_index()

In [7]:
one_order_customers

,customer_unique_id,orders
0,555ecf6bf501a3448c741080fc5b578b,1
1,516557ef1dc15a5f71608f273fae7b0b,1
2,54a6ac9eafde53270240650b842f57ef,1
3,516cfa9d1d99affadc1e170664de689f,1
4,51848a3b6669c501bf780100dcd9e7c2,1
...,...,...
93094,a925c3e5df82fdc6082f1383d2834998,1
93095,a92605ec492805540520d3a73aaeeb6e,1
93096,a926aae38267e7f54e67de9b5775d0a5,1
93097,a926cfc9bc7b082335de50450f48eec9,1


Количество покупателей, совершивших одну покупку равно 93099.

2. Сколько заказов в месяц в среднем не доставляется по разным причинам (вывести детализацию по причинам)?

In [8]:
not_delivered = orders.query('order_status != "delivered"') # Посмотрим на недоставленные заказы

In [9]:
not_delivered['month'] = not_delivered['order_purchase_timestamp'].dt.month

/opt/tljh/user/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
not_delivered.groupby(['month', 'order_status'], as_index = False) \
    .agg({'order_id': 'count'}) \
    .pivot(index = 'month', columns = 'order_status', values = 'order_id') \
    .fillna(0)

order_status,approved,canceled,created,invoiced,processing,shipped,unavailable
month,,,,,,,
1,0.0,37.0,0.0,27.0,38.0,90.0,58.0
2,1.0,90.0,1.0,17.0,38.0,78.0,75.0
3,0.0,59.0,0.0,26.0,32.0,178.0,49.0
4,1.0,33.0,0.0,28.0,18.0,148.0,14.0
5,0.0,53.0,0.0,40.0,29.0,109.0,47.0
6,0.0,34.0,0.0,14.0,12.0,90.0,28.0
7,0.0,69.0,0.0,20.0,12.0,116.0,70.0
8,0.0,111.0,0.0,43.0,18.0,88.0,39.0
9,0.0,37.0,0.0,17.0,22.0,40.0,38.0


Получили количество недоставленных заказов по месяцам.

3. По каждому товару определить, в какой день недели товар чаще всего покупается.

In [12]:
df = pd.read_csv('/home/jupyter-a-berezin-18/Project 1/olist_order_items_dataset.csv')

In [13]:
items = df.merge(orders, how = 'inner', on = 'order_id')

In [14]:
items['weekday'] = items['order_purchase_timestamp'].dt.weekday # 0 - понедельник, 6 - воскресенье

In [15]:
def weekday(n):
    if n == 0:
        return 'Monday'
    elif n == 1:
        return 'Tuesday'
    elif n == 2:
        return 'Wednesday'
    elif n == 3:
        return 'Thursday'
    elif n == 4:
        return 'Friday'
    elif n == 5:
        return 'Saturday'
    elif n == 6:
        return 'Sunday'

In [16]:
items['weekday'] = items.weekday.apply(weekday)

In [17]:
day = items.groupby(['product_id', 'weekday'], as_index = False) \
    .agg({'order_id': 'count'})

In [18]:
day.loc[day.groupby(['product_id'], sort=False)['order_id'].idxmax()][['product_id', 'weekday', 'order_id']] \
    .reset_index() \
    .drop(columns = {'index'})

,product_id,weekday,order_id
0,00066f42aeeb9f3007548bb9d3f33c38,Sunday,1
1,00088930e925c41fd95ebfe695fd2655,Tuesday,1
2,0009406fd7479715e4bef61dd91f2462,Thursday,1
3,000b8f95fcb9e0096488278317764d19,Friday,1
4,000d9be29b5207b54e86aa1b1ac54872,Tuesday,1
...,...,...,...
32946,fff6177642830a9a94a0f2cba5e476d1,Saturday,1
32947,fff81cc3158d2725c0655ab9ba0f712c,Monday,1
32948,fff9553ac224cec9d15d49f5a263411f,Friday,1
32949,fffdb2d0ec8d6a61f0a0a0db3f25b441,Tuesday,2


4. Сколько у каждого из пользователей в среднем покупок в неделю (по месяцам)? Не стоит забывать, что внутри месяца может быть не целое количество недель. Например, в ноябре 2021 года 4,28 недели. И внутри метрики это нужно учесть. 

In [19]:
users = merge.groupby(['customer_unique_id', 'order_purchase_timestamp']) \
    .agg({'order_id':'count'}) \
    .sort_values('order_id') \
    .reset_index()

In [20]:
users

,customer_unique_id,order_purchase_timestamp,order_id
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1
1,aa45ad6fcba95488ece6beda576c7106,2017-04-15 14:45:34,1
2,aa45ad0284650e38707f49e82b230f69,2018-03-29 22:10:38,1
3,aa456eb5182e9ce08561156920c2af2a,2017-09-26 16:55:16,1
4,aa4551527eecfd02e170883a059ca155,2017-09-11 23:46:26,1
...,...,...,...
99144,7a443ff6dbc48349835cb18a9355ea1b,2017-07-25 13:52:07,2
99145,84f30d2fd3937449dc7e0c835c281961,2018-07-18 17:13:02,2
99146,1bbf6de8bbe287cb474e57d437d72969,2017-10-04 19:40:44,2
99147,66980c3775537536f77b434d74e520f5,2018-03-31 15:08:21,3


In [21]:
users['number_of_weeks'] = round((users.order_purchase_timestamp.dt.daysinmonth)/7, 2)

In [22]:
users['orders_per_week'] = users['order_id']/ users['number_of_weeks']

In [23]:
users

,customer_unique_id,order_purchase_timestamp,order_id,number_of_weeks,orders_per_week
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,4.43,0.225734
1,aa45ad6fcba95488ece6beda576c7106,2017-04-15 14:45:34,1,4.29,0.233100
2,aa45ad0284650e38707f49e82b230f69,2018-03-29 22:10:38,1,4.43,0.225734
3,aa456eb5182e9ce08561156920c2af2a,2017-09-26 16:55:16,1,4.29,0.233100
4,aa4551527eecfd02e170883a059ca155,2017-09-11 23:46:26,1,4.29,0.233100
...,...,...,...,...,...
99144,7a443ff6dbc48349835cb18a9355ea1b,2017-07-25 13:52:07,2,4.43,0.451467
99145,84f30d2fd3937449dc7e0c835c281961,2018-07-18 17:13:02,2,4.43,0.451467
99146,1bbf6de8bbe287cb474e57d437d72969,2017-10-04 19:40:44,2,4.43,0.451467
99147,66980c3775537536f77b434d74e520f5,2018-03-31 15:08:21,3,4.43,0.677201


5. Напиши функцию на python, позволяющую строить когортный анализ. В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц.

In [24]:
def get_cohort(customers, orders):
    df = orders.merge(customers, how = 'inner', on = 'customer_id')
    
    df = df[['customer_unique_id', 'order_id', 'order_purchase_timestamp']].drop_duplicates()
    
    df['order_month'] = df['order_purchase_timestamp'].dt.to_period('M')
    
    df['cohort'] = df.groupby('customer_unique_id')['order_purchase_timestamp'] \
                 .transform('min') \
                 .dt.to_period('M') 
    
    df_cohort = df.groupby(['cohort', 'order_month']) \
              .agg(n_customers=('customer_unique_id', 'nunique')) \
              .reset_index(drop=False)
    
    df_cohort['number'] = (df_cohort.order_month - df_cohort.cohort).apply(attrgetter('n'))
    
    cohort_pivot = df_cohort.pivot_table(index = 'cohort',
                                     columns = 'number',
                                     values = 'n_customers')
    
    cohort_size = cohort_pivot.iloc[:,0]
    retention_matrix = cohort_pivot.divide(cohort_size, axis = 0)
    
    ur_style = (retention_matrix
            .style
            .set_caption('User retention by cohort')
            .background_gradient(cmap='viridis') 
            .highlight_null('white') 
            .format("{:.2%}", na_rep="")) 

    return {"retention_matrix": retention_matrix,
            "style": ur_style}

In [25]:
cohort = get_cohort(customers, orders)

In [26]:
cohort['style']

number,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20
cohort,,,,,,,,,,,,,,,,,,,,
2016-09,100.00%,,,,,,,,,,,,,,,,,,,
2016-10,100.00%,,,,,,0.31%,,,0.31%,,0.31%,,0.31%,,0.31%,,0.31%,0.62%,0.62%
2016-12,100.00%,100.00%,,,,,,,,,,,,,,,,,,
2017-01,100.00%,0.39%,0.26%,0.13%,0.39%,0.13%,0.52%,0.13%,0.13%,,0.39%,0.13%,0.79%,0.39%,0.13%,0.13%,0.26%,0.39%,0.13%,
2017-02,100.00%,0.23%,0.29%,0.11%,0.40%,0.11%,0.23%,0.17%,0.17%,0.23%,0.11%,0.29%,0.17%,0.17%,0.11%,0.06%,0.06%,0.23%,,
2017-03,100.00%,0.49%,0.38%,0.38%,0.34%,0.15%,0.15%,0.30%,0.34%,0.08%,0.38%,0.15%,0.23%,0.11%,0.15%,0.23%,0.08%,0.15%,,
2017-04,100.00%,0.60%,0.21%,0.17%,0.34%,0.26%,0.34%,0.30%,0.30%,0.17%,0.26%,0.09%,0.09%,0.04%,0.09%,0.09%,0.21%,,,
2017-05,100.00%,0.50%,0.50%,0.39%,0.31%,0.33%,0.42%,0.17%,0.25%,0.31%,0.25%,0.33%,0.25%,0.03%,0.19%,0.25%,,,,
2017-06,100.00%,0.48%,0.35%,0.41%,0.25%,0.38%,0.38%,0.22%,0.13%,0.22%,0.32%,0.35%,0.16%,0.13%,0.19%,,,,,


В период с января по декабрь выяви когорту с самым высоким retention на 3-й месяц.

Т.к период с января по декабрь представлен полностью только за 2017 год, выборем соответствующие когорты.

In [27]:
cohort['retention_matrix'].query('"2017-01" <= cohort <= "2017-12"')[3].idxmax()

Period('2017-06', 'M')

6. Построй RFM-кластеры для пользователей. Выведи для каждого кластера средние значения метрик R, F, M (и опиши подробно, как были построены метрики R, F, M). 

In [28]:
items = pd.read_csv('/home/jupyter-a-berezin-18/Project 1/olist_order_items_dataset.csv')

In [29]:
price_df = items.groupby('order_id').agg({'price': 'sum'})

In [30]:
orders_6 = merge.set_index('order_id')

In [31]:
order_payment = orders_6.join(price_df).reset_index()

In [32]:
order_payment

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,price
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,29.99
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,118.70
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,159.90
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,45.00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,19.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09 09:54:05,2017-03-09 09:54:05,2017-03-10 11:18:03,2017-03-17 15:08:01,2017-03-28,6359f309b166b0196dbf7ad2ac62bb5a,12209,sao jose dos campos,SP,72.00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06 12:58:58,2018-02-06 13:10:37,2018-02-07 23:22:42,2018-02-28 17:37:56,2018-03-02,da62f9e57a76d978d02ab5362c509660,11722,praia grande,SP,174.90
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27 14:46:43,2017-08-27 15:04:16,2017-08-28 20:52:26,2017-09-21 11:24:17,2017-09-27,737520a9aad80b3fbbdad19b66b37b30,45920,nova vicosa,BA,205.99
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08 21:28:27,2018-01-08 21:36:21,2018-01-12 15:35:03,2018-01-25 23:32:54,2018-02-15,5097a5312c8b157bb7be58ae360ef43c,28685,japuiba,RJ,359.98


Примем что:  
- Recency — разница между текущей датой и датой последнего заказа (order_purchase_timestamp)  
- Frequency — количество транзакций  
- Monetary — сумма покупок

За текущую дату примем самую свежую + 1

In [33]:
last_date = order_payment['order_purchase_timestamp'].max() + dt.timedelta(days=1)

Сгруппируем по customer_unique_id и посчитаем recency, frequency, monetary_value

In [34]:
rfm_table = order_payment.groupby('customer_unique_id') \
    .agg({'order_purchase_timestamp': lambda x: (last_date - x.max()).days,
                                              'order_id': lambda x: len(x),
                                              'price': lambda x: x.sum()})

In [35]:
rfm_table.rename(columns={'order_purchase_timestamp': 'recency', 
                          'order_id': 'frequency', 
                          'price': 'monetary_value'}, inplace=True)

In [36]:
rfm_table

,recency,frequency,monetary_value
customer_unique_id,,,
0000366f3b9a7992bf8c76cfdf3221e2,161,1,129.90
0000b849f77a49e4a4ce2b2a4ca5be3f,164,1,18.90
0000f46a3911fa3c0805444483337064,586,1,69.00
0000f6ccb0745a6a4b88665a16c9f078,370,1,25.99
0004aac84e0df4da2b147fca70cf8255,337,1,180.00
...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,496,1,1570.00
fffea47cd6d3cc0a88bd621562a9d061,311,1,64.89
ffff371b4d645b6ecea244b27531430a,618,1,89.90


Разделим на 5 сегментов

In [37]:
quantiles = rfm_table.quantile(q=[0.20, 0.40, 0.60, 0.80])
quantiles = quantiles.to_dict()
segmented_rfm = rfm_table

def RScore(x,p,d):
     if x <= d[p][0.20]:
         return 1
     elif x <= d[p][0.40]:
         return 2
     elif x <= d[p][0.60]: 
         return 3
     elif x<=d[p][0.80]:
         return 4
     else:
         return 5

Присвоим каждому клиенту RScore

In [38]:
segmented_rfm['r_quartile'] = segmented_rfm['recency'].apply(RScore, args=('recency',quantiles,))
segmented_rfm['f_quartile'] = segmented_rfm['frequency'].apply(RScore, args=('frequency',quantiles,))
segmented_rfm['m_quartile'] = segmented_rfm['monetary_value'].apply(RScore, args=('monetary_value',quantiles,))
segmented_rfm['RFMScore'] = segmented_rfm.r_quartile.map(str) + segmented_rfm.f_quartile.map(str) + segmented_rfm.m_quartile.map(str)

In [39]:
RFM = segmented_rfm.reset_index()

In [46]:
RFM

,customer_unique_id,recency,frequency,monetary_value,r_quartile,f_quartile,m_quartile,RFMScore
0,0000366f3b9a7992bf8c76cfdf3221e2,161,1,129.90,2,1,4,214
1,0000b849f77a49e4a4ce2b2a4ca5be3f,164,1,18.90,2,1,1,211
2,0000f46a3911fa3c0805444483337064,586,1,69.00,5,1,2,512
3,0000f6ccb0745a6a4b88665a16c9f078,370,1,25.99,4,1,1,411
4,0004aac84e0df4da2b147fca70cf8255,337,1,180.00,4,1,5,415
...,...,...,...,...,...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,496,1,1570.00,5,1,5,515
96092,fffea47cd6d3cc0a88bd621562a9d061,311,1,64.89,3,1,2,312
96093,ffff371b4d645b6ecea244b27531430a,618,1,89.90,5,1,3,513
96094,ffff5962728ec6157033ef9805bacc48,169,1,115.00,2,1,4,214


Посчитаем средние значения для каждого кластера

In [40]:
RFM_segments = RFM.groupby('RFMScore') \
.agg({'recency': 'mean', 'frequency': 'mean', 'monetary_value': ['mean', 'count']}) \
.round(1)

In [43]:
RFM_segments

recency frequency monetary_value      
            mean      mean           mean count
RFMScore                                       
111         94.4       1.0           25.1  3848
112         93.4       1.0           53.2  3764
113         95.4       1.0           88.2  3694
114         96.0       1.0          139.3  3860
115         97.0       1.0          413.8  3466
151         87.2       2.0           24.4    20
152         84.3       2.0           55.8    53
153         94.2       2.1           90.1    93
154         96.9       2.0          143.5   161
155         94.8       2.3          414.7   365
211        186.8       1.0           25.1  3639
212        187.3       1.0           52.5  3650
213        184.9       1.0           88.2  3836
214        186.8       1.0          140.0  3975
215        186.5       1.0          411.6  3577
251        186.3       2.0           30.7    17
252        180.0       2.0           57.8    44
253        187.4       2.0           90.3    85
254        187.3       2.1          144.9   160
255        185.5       2.2          414.3   316
311        269.7       1.0           24.3  3901
312        271.1       1.0           52.5  3590
313        272.4       1.0           88.2  3912
314        271.6       1.0          141.1  3682
315        271.6       1.0          370.3  3329
351        250.9       2.0           28.4    11
352        269.6       2.0           55.9    44
353        270.2       2.1           90.0    79
354        267.3       2.0          147.0   171
355        270.3       2.1          395.0   324
411        366.4       1.0           24.0  3737
412        368.4       1.0           53.1  4088
413        367.6       1.0           88.4  3807
414        367.0       1.0          141.0  3494
415        367.5       1.0          415.5  3583
451        368.1       2.0           21.0    17
452        363.0       2.0           53.7    40
453        366.0       2.0           89.2   108
454        367.8       2.1          140.7   141
455        371.1       2.1          427.1   285
511        529.4       1.0           23.5  4083
512        521.8       1.0           53.4  3928
513        522.8       1.0           89.0  3784
514        525.7       1.0          140.0  3285
515        529.1       1.0          407.3  3587
551        548.0       2.2           25.4    21
552        530.6       2.2           53.3    49
553        513.1       2.0           87.7    69
554        515.2       2.0          142.3   122
555        510.4       2.1          417.7   202